# Imports


In [1]:
import pandas as pd
import os
%matplotlib inline 

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

from scipy import stats

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import cross_val_score

# Read data

In [2]:
# import opendatasets as od
# dataset = 'https://www.kaggle.com/competitions/child-mind-institute-problematic-internet-use/data'
# od.download(dataset)

In [3]:
df = pd.read_csv("/home/ku18/CS412/dataset/train.csv")
df.columns

Index(['id', 'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
       'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
       'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST'

In [4]:
TARGET_COLS = [
    # "PCIAT-PCIAT_01",
    # "PCIAT-PCIAT_02",
    # "PCIAT-PCIAT_03",
    # "PCIAT-PCIAT_04",
    # "PCIAT-PCIAT_05",
    # "PCIAT-PCIAT_06",
    # "PCIAT-PCIAT_07",
    # "PCIAT-PCIAT_08",
    # "PCIAT-PCIAT_09",
    # "PCIAT-PCIAT_10",
    # "PCIAT-PCIAT_11",
    # "PCIAT-PCIAT_12",
    # "PCIAT-PCIAT_13",
    # "PCIAT-PCIAT_14",
    # "PCIAT-PCIAT_15",
    # "PCIAT-PCIAT_16",
    # "PCIAT-PCIAT_17",
    # "PCIAT-PCIAT_18",
    # "PCIAT-PCIAT_19",
    # "PCIAT-PCIAT_20",
    # "PCIAT-PCIAT_Total",
    "sii"
]

FEATURE_COLS = [
    "Basic_Demos-Age",
    "Basic_Demos-Sex",
    "CGAS-CGAS_Score",
    "Physical-BMI",
    "Physical-Height",
    "Physical-Weight",
    "Physical-Waist_Circumference",
    "Physical-Diastolic_BP",
    "Physical-HeartRate",
    "Physical-Systolic_BP",
    "Fitness_Endurance-Max_Stage",
    "Fitness_Endurance-Time_Mins",
    "Fitness_Endurance-Time_Sec",
    "FGC-FGC_CU",
    "FGC-FGC_CU_Zone",
    "FGC-FGC_GSND",
    "FGC-FGC_GSND_Zone",
    "FGC-FGC_GSD",
    "FGC-FGC_GSD_Zone",
    "FGC-FGC_PU",
    "FGC-FGC_PU_Zone",
    "FGC-FGC_SRL",
    "FGC-FGC_SRL_Zone",
    "FGC-FGC_SRR",
    "FGC-FGC_SRR_Zone",
    "FGC-FGC_TL",
    "FGC-FGC_TL_Zone",
    "BIA-BIA_Activity_Level_num",
    "BIA-BIA_BMC",
    "BIA-BIA_BMI",
    "BIA-BIA_BMR",
    "BIA-BIA_DEE",
    "BIA-BIA_ECW",
    "BIA-BIA_FFM",
    "BIA-BIA_FFMI",
    "BIA-BIA_FMI",
    "BIA-BIA_Fat",
    "BIA-BIA_Frame_num",
    "BIA-BIA_ICW",
    "BIA-BIA_LDM",
    "BIA-BIA_LST",
    "BIA-BIA_SMM",
    "BIA-BIA_TBW",
    "PAQ_A-PAQ_A_Total",
    "PAQ_C-PAQ_C_Total",
    "SDS-SDS_Total_Raw",
    "SDS-SDS_Total_T",
    "PreInt_EduHx-computerinternet_hoursday",
]
data = df[FEATURE_COLS]
target = df[TARGET_COLS]

In [5]:
len(data)

3960

# Missing values in data

In [6]:
print(data.isna().sum())

Basic_Demos-Age                              0
Basic_Demos-Sex                              0
CGAS-CGAS_Score                           1539
Physical-BMI                               938
Physical-Height                            933
Physical-Weight                            884
Physical-Waist_Circumference              3062
Physical-Diastolic_BP                     1006
Physical-HeartRate                         993
Physical-Systolic_BP                      1006
Fitness_Endurance-Max_Stage               3217
Fitness_Endurance-Time_Mins               3220
Fitness_Endurance-Time_Sec                3220
FGC-FGC_CU                                1638
FGC-FGC_CU_Zone                           1678
FGC-FGC_GSND                              2886
FGC-FGC_GSND_Zone                         2898
FGC-FGC_GSD                               2886
FGC-FGC_GSD_Zone                          2897
FGC-FGC_PU                                1650
FGC-FGC_PU_Zone                           1689
FGC-FGC_SRL  

## Iterative imputation to fill in missing values in the dataset|

In [ ]:
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer

iterative_imputer = IterativeImputer(max_iter=10, random_state=0)

data = pd.DataFrame(iterative_imputer.fit_transform(data))
data.columns = FEATURE_COLS

# Missing values in target

In [ ]:
print(target.isna().sum())

## drop missing target values

In [ ]:
df = pd.concat([data, target], axis = 1)

In [ ]:
df = df.dropna()

In [ ]:
print(len(df))

# Classification models

In [ ]:
X = df[FEATURE_COLS]
y = df[TARGET_COLS]
y = y.values.flatten()
print(len(X))
print(len(y))

## Classifiers

In [ ]:
# create the model: Random Forest
myRF = RandomForestClassifier(max_depth=15, random_state=0, n_estimators=100)

# Perform k-fold cross-validation
accuracyRF = cross_val_score(myRF,X,y,cv=10)

# report accuracy in each fold, and mean accuracy
print(accuracyRF)
print(np.mean(accuracyRF))

In [ ]:
# create the model: XGBoost
myXGB = XGBClassifier(max_depth=15)

# Perform k-fold cross-validation
accuracyXGB = cross_val_score(myXGB,X,y,cv=10)

# report accuracy in each fold, and mean accuracy
print(accuracyXGB)
print(np.mean(accuracyXGB))

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

mySVC = make_pipeline(StandardScaler(), SVC(kernel='linear', random_state=42))
accuracySVC = cross_val_score(mySVC,X,y,cv=10)
print(accuracySVC)
print(np.mean(accuracySVC))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

# Example data setup (replace with your actual data)
# X = pd.DataFrame(...)  # Your feature matrix
# y = pd.Series(...)     # Your target variable

# Preprocess the data
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the FFN model
model = keras.Sequential()
model.add(layers.Input(shape=(X_train.shape[1],)) )  # Input layer
model.add(layers.Dense(64, activation='relu'))         # First hidden layer
model.add(layers.Dense(64, activation='relu'))         # Second hidden layer
model.add(layers.Dense(1, activation='sigmoid'))       # Output layer for binary classification
# For multi-class classification, use:
# model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use 'categorical_crossentropy' for multi-class
              metrics=['accuracy'])
 
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy:.4f}')

# Optionally, plot the training history
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
